In [2]:
# імпорт бібліотек
import matplotlib.pyplot as plt
import cv2
import os
from matplotlib.image import imread
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.preprocessing import image_dataset_from_directory
import dlib


## Глобальні змінні

In [9]:
# директорія сирих картинки без обрізки
input_folder_path = "./raw_dataset"

# директорії картинок для різних завдань
output_folder_path = "./cutted_dataset"
output_folder_path_hog = "./cutted_dataset_hog"
output_folder_path_hog_eyes = "./finded_eyes_hog"
output_folder_smiles_path = "./finded_smiles"

# Загрузка каскада Хаара для обнаружения облич
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Загрузка каскада Хаара для обнаружения усмішок
smile_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_smile.xml"
)

# ініціалізація моделі hog
hog_face_detector = dlib.get_frontal_face_detector()
# ініціалізація моделі для пошуку очей
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

## Допоміжні функції 

In [4]:
def extract_data_from_dataset(dataset):
    """
    Функція для витягування зображень та міток з датасету.

    Параметри:
    - `dataset:` TensorFlow датасет, який містить пари (зображення, мітка).

    Повертає:
    - x: `Масив NumPy`, що містить зображення.
    - y: `Масив NumPy`, що містить мітки.
    """
    x_data = []
    y_data = []

    for images, labels in dataset:
        x_data.append(images.numpy())
        y_data.append(labels.numpy())

    x_data = np.concatenate(x_data)
    y_data = np.concatenate(y_data)

    return x_data, y_data

def apply_pca_to_images(images, n_components=100):
    """
    Функція для застосування методу головних компонентів до зображень.

    Параметри:
    - `images`: Масив NumPy, який містить зображення (форма: (кількість_зразків, розмір_зображення_1, розмір_зображення_2, кількість_каналів)).
    - `n_components`: Кількість головних компонент.

    Повертає:
    - `metric_pca`: Масив NumPy, що містить зображення у просторі головних компонент.
    """
    # Змінення форми images в (кількість_зразків, кількість_ознак)
    metric_reshaped = images.reshape(images.shape[0], -1)

    # Ініціалізація PCA та виконання PCA
    pca = PCA(n_components=n_components)
    metric_pca = pca.fit_transform(metric_reshaped)

    return metric_pca

def crop_faces(image_path, output_path):
    """
    Обрізає обличчя на зображенні та зберігає його.

    Параметри:
    - `image_path`: Шлях до вхідного зображення.
    - `output_path`: Шлях для зберігання обрізаного обличчя.

    Повертає:
    - `Відсутнє` (зображення зберігається у вказаному вихідному шляху).
    """
    # Чтение изображения
    img = cv2.imread(image_path)
    # Преобразование в градации серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Обнаружение лиц на изображении
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.8, minNeighbors=10)

    for (x, y, w, h) in faces:
        # Обрезка изображения по координатам лица
        face = img[y:y+h, x:x+w]
        # Сохранение обрезанного изображения в той же подпапке
        output_file = os.path.join(output_path, os.path.basename(image_path))
        cv2.imwrite(output_file, face)    
            
def crop_faces_hog(image_path, output_path):
    """
    Обрізає обличчя за допомогою HOG-детектора та зберігає його.

    Параметри:
    - `image_path`: Шлях до вхідного зображення.
    - `output_path`: Шлях для зберігання обрізаного обличчя.

    Повертає:
    -`Відсутнє` (зображення обличчя зберігається у вказаному вихідному шляху).
    """
    # Чтение изображения
    img = cv2.imread(image_path)
    # Преобразование в градации серого
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB);

    # Обнаружение лиц на изображении
    faces = hog_face_detector(img_rgb)

    if faces:
        for i, face in enumerate(faces):
            x, y, w, h = face.left(), face.top(), face.width(), face.height()

            if 0 <= x < img.shape[1] and 0 <= y < img.shape[0] and 0 <= x + w < img.shape[1] and 0 <= y + h < img.shape[0]:
                face_image = img_rgb[y:y+h, x:x+w]
                output_file = os.path.join(output_path, os.path.basename(image_path))   
                cv2.imwrite(output_file, cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR))
                                        
def find_eyes_hog(image_path, output_path):
    """
    Знаходить координати очей на зображенні за допомогою HOG-детектора та зберігає з ними прямокутники.

    Параметри:
    - `image_path`: Шлях до вхідного зображення.
    - `output_path`: Шлях для зберігання зображення з нанесеними прямокутниками навколо очей.

    Повертає:
    -`Відсутнє` (зображення з прямокутниками навколо очей зберігається у вказаному вихідному шляху).
    """
    # Чтение изображения
    img = cv2.imread(image_path)
    # Преобразование в градации серого
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB);

    # Обнаружение лиц на изображении
    faces = hog_face_detector(img_rgb)

    for i, face in enumerate(faces):
                landmarks = predictor(img_rgb, face)

                left_eye_x = landmarks.part(36).x
                left_eye_y = landmarks.part(37).y
                right_eye_x = landmarks.part(39).x
                right_eye_y = landmarks.part(40).y

                left_eye_width = right_eye_x - left_eye_x
                left_eye_height = right_eye_y - left_eye_y
                cv2.rectangle(img_rgb, (left_eye_x, left_eye_y), (left_eye_x + left_eye_width, left_eye_y + left_eye_height), (0, 255, 0), 2)

                left_eye_x = landmarks.part(42).x
                left_eye_y = landmarks.part(43).y
                right_eye_x = landmarks.part(45).x
                right_eye_y = landmarks.part(46).y

                right_eye_width = right_eye_x - left_eye_x
                right_eye_height = right_eye_y - left_eye_y
                cv2.rectangle(img_rgb, (left_eye_x, left_eye_y), (left_eye_x + right_eye_width, left_eye_y + right_eye_height), (0, 255, 0), 2)

                output_file = os.path.join(output_path, os.path.basename(image_path))
                cv2.imwrite(output_file, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))
   
def find_smiles(image_path, output_path):
    """
    Знаходить усмішки на зображенні та зберігає з ними прямокутники.

    Параметри:
    - `image_path`: Шлях до вхідного зображення.
    - `output_path`: Шлях для зберігання зображення з нанесеними прямокутниками навколо усмішок.

    Повертає:
    -`Відсутнє` (зображення з прямокутниками навколо усмішок зберігається у вказаному вихідному шляху).
    """
    # Чтение изображения
    img = cv2.imread(image_path)
    # Преобразование в градации серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Обнаружение улыбок на изображении
    smiles = smile_cascade.detectMultiScale(gray, scaleFactor=1.8, minNeighbors=20)

    for (sx, sy, sw, sh) in smiles:
        cv2.rectangle(img, (sx, sy), (sx+sw, sy+sh), (0, 255, 0), 2)

    output_file = os.path.join(output_path, os.path.basename(image_path))
    cv2.imwrite(output_file, img)


## <span style="color:lightblue">`Завдання 1`. Реалізація обрізки методом Віоли Джонса.</span>


##### `1.` Спочатку треба обрати датасет з мітками картинок. Я обрав датасет [Celebrity Face Image Dataset](https://www.kaggle.com/datasets/vishesh1412/celebrity-face-image-dataset/). Після чого провів обрізку


In [4]:
# Перебор всех файлов в папке
for root, dirs, files in os.walk(input_folder_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(root, file)
            # Создание подпапки в output_folder, если она не существует
            output_subfolder = os.path.join(output_folder_path, os.path.relpath(root, input_folder_path))
            os.makedirs(output_subfolder, exist_ok=True)
            # Обрезка лиц на изображении и сохранение в той же подпапке
            crop_faces(image_path, output_subfolder)

print("Обрезка завершена.")

Обрезка завершена.


##### `2.` Розбиваю датасет на test та train вибірки.

In [11]:
train_ds, test_ds = image_dataset_from_directory(
    output_folder_path,
    validation_split = 0.2,
    subset='both',
    label_mode='categorical',
    batch_size = 1,
    seed=0,
)


Found 896 files belonging to 17 classes.
Using 717 files for training.
Using 179 files for validation.


##### `3.` Достаю з вибірки дані (картинки та класи) для тестової та навчальної вибірки. Також застосовую PCA для оптимізації роботи алгоритмів.

In [12]:
x_train, y_train = extract_data_from_dataset(train_ds)

In [13]:
x_test, y_test = extract_data_from_dataset(test_ds)

In [14]:
x_train_pca = apply_pca_to_images(x_train, 100)

In [15]:
x_test_pca=apply_pca_to_images(x_test,100)

##### `4.` Розпізнавання картинки знаменитості методом `RandomForestClassifier`.

In [16]:
# Ініціалізація та навчання RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train_pca, y_train)

# Передбачення на тестовому наборі
y_pred_train = clf.predict(x_train_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_train, y_pred_train)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_train, y_pred_train, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_train, y_pred_train, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 0.99860529986053
Precision: 0.99860529986053
Recall: 0.99860529986053


In [18]:
# Передбачення на тестовому наборі
y_pred_test = clf.predict(x_test_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_test, y_pred_test)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_test, y_pred_test, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_test, y_pred_test, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 0.0
Precision: 0.0
Recall: 0.0


/home/kyrylo/.cache/pypoetry/virtualenvs/python-technicalvision-labs-XHQBv0MF-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### `5.` Розпізнавання картинки знаменитості методом `MLPClassifier`.

In [19]:
# Ініціалізація та навчання MLPClassifier
clf = MLPClassifier()
clf.fit(x_train_pca, y_train)

# Передбачення на тестовому наборі
y_pred_train = clf.predict(x_train_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_train, y_pred_train)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_train, y_pred_train, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_train, y_pred_train, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [20]:
# Передбачення на тестовому наборі
y_pred_test = clf.predict(x_test_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_test, y_pred_test)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_test, y_pred_test, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_test, y_pred_test, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 0.0223463687150838
Precision: 0.0223463687150838
Recall: 0.0223463687150838


##### `6.` Розпізнавання картинки знаменитості методом `SVM-метод опорних векторів`.

In [21]:
# Використання лише першого стовпця для one-hot encoding як мітки
y_train_single = y_train.argmax(axis=1)

# Ініціалізація та навчання SVC
clf = SVC()
clf.fit(x_train_pca, y_train_single)

# Передбачення на тренувальному наборі
y_pred_train = clf.predict(x_train_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_train_single, y_pred_train)
print(f'Accuracy: {accuracy}')

# Оцінка precision та recall
precision = precision_score(y_train_single, y_pred_train, average="micro")
print(f'Precision: {precision}')
recall = recall_score(y_train_single, y_pred_train, average="micro")
print(f'Recall: {recall}')

Accuracy: 0.8228730822873083
Precision: 0.8228730822873083
Recall: 0.8228730822873083


In [22]:
# Використання лише першого стовпця для one-hot encoding як мітки
y_test_single = y_test.argmax(axis=1)

# Передбачення на тренувальному наборі
y_pred_test = clf.predict(x_test_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_test_single, y_pred_test)
print(f'Accuracy: {accuracy}')

# Оцінка precision та recall
precision = precision_score(y_test_single, y_pred_test, average="micro")
print(f'Precision: {precision}')
recall = recall_score(y_test_single, y_pred_test, average="micro")
print(f'Recall: {recall}')

Accuracy: 0.061452513966480445
Precision: 0.061452513966480445
Recall: 0.061452513966480445


## <span style="color:lightblue">`Завдання 2`. Реалізація обрізки методом Hog детектор.</span>

##### `1.` Обрізка картинок методом HOG.

In [7]:
hog_face_detector = dlib.get_frontal_face_detector()

# Перебор всех файлов в папке
for root, dirs, files in os.walk(input_folder_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(root, file)
            # Создание подпапки в output_folder, если она не существует
            output_subfolder = os.path.join(output_folder_path_hog, os.path.relpath(root, input_folder_path))
            os.makedirs(output_subfolder, exist_ok=True)
            # Обрезка лиц на изображении и сохранение в той же подпапке
            crop_faces_hog(image_path, output_subfolder)

print("Обрезка завершена.")

Обрезка завершена.


##### `2.` Достаю з вибірки дані (картинки та класи) для тестової та навчальної вибірки. Також застосовую PCA для оптимізації роботи алгоритмів.

In [14]:
train_ds_hog, test_ds_hog = image_dataset_from_directory(
    output_folder_path_hog,
    validation_split = 0.2,
    subset='both',
    label_mode='categorical',
    batch_size = 1,
    seed=0,
)

Found 1718 files belonging to 17 classes.
Using 1375 files for training.
Using 343 files for validation.


In [18]:
x_train_hog, y_train_hog = extract_data_from_dataset(train_ds_hog)

In [17]:
x_test_hog, y_test_hog = extract_data_from_dataset(test_ds_hog)

In [19]:
x_train_hog_pca = apply_pca_to_images(x_train_hog, 100)

In [ ]:
x_test_hog_pca = apply_pca_to_images(x_test_hog, 100)

##### `3.` Розпізнавання картинки знаменитості методом `RandomForestClassifier`.

In [22]:
# Ініціалізація та навчання RandomForestClassifier
clf_hog = RandomForestClassifier()
clf_hog.fit(x_train_hog_pca, y_train_hog)

# Передбачення на тестовому наборі
y_hog_pred_train = clf_hog.predict(x_train_hog_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_train_hog, y_hog_pred_train)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_train_hog, y_hog_pred_train, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_train_hog, y_hog_pred_train, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 0.9985454545454545
Precision: 0.9985454545454545
Recall: 0.9985454545454545


In [26]:
# Передбачення на тестовому наборі
y_hog_pred_test = clf_hog.predict(x_test_hog_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_test_hog, y_hog_pred_test)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_test_hog, y_hog_pred_test, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_test_hog, y_hog_pred_test, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 0.0
Precision: 0.0
Recall: 0.0


/home/kyrylo/.cache/pypoetry/virtualenvs/python-technicalvision-labs-XHQBv0MF-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### `4.` Розпізнавання картинки знаменитості методом `MLPClassifier`.

In [27]:
# Ініціалізація та навчання MLPClassifier
clf_hog = MLPClassifier()
clf_hog.fit(x_train_hog_pca, y_train_hog)

# Передбачення на тестовому наборі
y_hog_pred_train = clf_hog.predict(x_train_hog_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_train_hog, y_hog_pred_train)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_train_hog, y_hog_pred_train, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_train_hog, y_hog_pred_train, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [28]:
# Передбачення на тестовому наборі
y_hog_pred_test = clf_hog.predict(x_test_hog_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_test_hog, y_hog_pred_test)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_test_hog, y_hog_pred_test, average="micro")
print(f'Precision: {accuracy}')
recall = recall_score(y_test_hog, y_hog_pred_test, average="micro")
print(f'Recall: {accuracy}')

Accuracy: 0.026239067055393587
Precision: 0.026239067055393587
Recall: 0.026239067055393587


##### `5.` Розпізнавання картинки знаменитості методом `SVM-метод опорних векторів`.

In [29]:
# Використання лише першого стовпця для one-hot encoding як мітки
y_train_single = y_train_hog.argmax(axis=1)

# Ініціалізація та навчання SVC
clf_hog = SVC()
clf_hog.fit(x_train_hog_pca, y_train_single)

# Передбачення на тренувальному наборі
y_hog_pred_train = clf_hog.predict(x_train_hog_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_train_single, y_hog_pred_train)
print(f'Accuracy: {accuracy}')

# Оцінка precision та recall
precision = precision_score(y_train_single, y_hog_pred_train, average="micro")
print(f'Precision: {precision}')
recall = recall_score(y_train_single, y_hog_pred_train, average="micro")
print(f'Recall: {recall}')

Accuracy: 0.808
Precision: 0.808
Recall: 0.808


In [30]:
# Використання лише першого стовпця для one-hot encoding як мітки
y_test_single = y_test_hog.argmax(axis=1)

# Передбачення на тренувальному наборі
y_hog_pred_test = clf_hog.predict(x_test_hog_pca)

# Оцінка точності моделі
accuracy = accuracy_score(y_test_single, y_hog_pred_test)
print(f'Accuracy: {accuracy}')

# Оцінка precision та recall
precision = precision_score(y_test_single, y_hog_pred_test, average="micro")
print(f'Precision: {precision}')
recall = recall_score(y_test_single, y_hog_pred_test, average="micro")
print(f'Recall: {recall}')

Accuracy: 0.15451895043731778
Precision: 0.15451895043731778
Recall: 0.15451895043731778


## <span style="color:lightblue">`Завдання 3`. Реалізація пошуку очей методом Hog детектор.</span>

In [33]:
# Перебор всех файлов в папке
for root, dirs, files in os.walk(input_folder_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(root, file)
            # Создание подпапки в output_folder, если она не существует
            output_subfolder = os.path.join(output_folder_path_hog_eyes, os.path.relpath(root, input_folder_path))
            os.makedirs(output_subfolder, exist_ok=True)
            # Обрезка лиц на изображении и сохранение в той же подпапке
            find_eyes_hog(image_path, output_subfolder)

print("Обрезка завершена.")

Обрезка завершена.


## <span style="color:lightblue">`Завдання 4`. Реалізація пошуку посмішок методом Віоли Джонса.</span>

In [40]:
# Перебор всех файлов в папке
for root, dirs, files in os.walk(input_folder_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(root, file)
            # Создание подпапки в output_folder, если она не существует
            output_subfolder = os.path.join(output_folder_smiles_path, os.path.relpath(root, input_folder_path))
            os.makedirs(output_subfolder, exist_ok=True)
            # Обрезка лиц на изображении и сохранение в той же подпапке
            find_smiles(image_path, output_subfolder)

print("Обрезка завершена.")

Обрезка завершена.
